In [4]:
import pandas as pd
import psycopg2
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('layoff1.csv', 'layoff1'),
    ('layoffs.csv', 'layoffs')    
]

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host='localhost',
    user='postgres',
    password='*******',
    dbname='Layoffs_db'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'C:/Users/Lenovo/Desktop/Sql/Project/Data'

# Function to get SQL data type
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INTEGER'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'TIMESTAMP'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame with proper encoding
    df = pd.read_csv(file_path, encoding='latin1')  # or use 'ISO-8859-1' if 'latin1' doesn't work
    
    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'"{col}" {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS "{table_name}" ({columns})'
    cursor.execute(create_table_query)
    
    # Save the DataFrame to a CSV without the index and with the same encoding
    temp_csv_path = os.path.join(folder_path, f'temp_{csv_file}')
    df.to_csv(temp_csv_path, index=False, header=False, encoding='latin1')
    
    # Use COPY to load the CSV data into the PostgreSQL table
    with open(temp_csv_path, 'r', encoding='latin1') as f:
        cursor.copy_expert(f'COPY "{table_name}" FROM STDIN WITH CSV', f)
    
    # Commit the transaction for the current CSV file
    conn.commit()
    
    # Remove the temporary CSV file
    os.remove(temp_csv_path)

# Close the connection
cursor.close()
conn.close()
